In [ ]:
# Model architecture inspired by Z. Hameed and B. Garcia-Zapirain, "Sentiment classification using a single-layered BiLSTM model", IEEE Access, vol. 8, pp. 73992-74001, 2020.

import gensim.downloader as api

info = api.info()  # show info about available models/datasets
w2v = api.load("word2vec-google-news-300") 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Bidirectional, LSTM, Input, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate

from keras.layers.embeddings import Embedding

def process_text(document):
     
    # Remove extra white space from text
    document = re.sub(r'\s+', ' ', document, flags=re.I)
         
    # Remove all the special characters from text
    document = re.sub(r'\W', ' ', str(document))
 
    return document

In [ ]:
df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

,Emotion,Statement
0,joy,"Thank you , Steven . I accept your advice ."
1,surprise,"Oh my God, I can't believe I have two-two chil..."
2,neutral,"Look, it's not that big of a deal, so Monica a..."
3,anger,And I wanna know why?!!
4,joy,"And, ah, you know, your fooling around with her."


In [ ]:
from tqdm import tqdm

df['preprocessedStatement'] = df.Statement.apply(process_text)
display(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Emotion,Statement,preprocessedStatement
0,joy,"Thank you , Steven . I accept your advice .",Thank you Steven I accept your advice
1,surprise,"Oh my God, I can't believe I have two-two chil...",Oh my God I can t believe I have two two chil...
2,neutral,"Look, it's not that big of a deal, so Monica a...",Look it s not that big of a deal so Monica a...
3,anger,And I wanna know why?!!,And I wanna know why
4,joy,"And, ah, you know, your fooling around with her.",And ah you know your fooling around with her


In [ ]:
max_length = df.preprocessedStatement.apply(lambda x: len(x.split())).max()

t = Tokenizer()
t.fit_on_texts(df['preprocessedStatement'] )
vocab_size = len(t.word_index) + 1
encoded_tweets = t.texts_to_sequences(df['preprocessedStatement'] )
X = pad_sequences(encoded_tweets, maxlen=max_length, padding='post')

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    try:
      embedding_vector = w2v[word]
    except KeyError:
      pass
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# Encode labels in column 'Emotion'. 
df['Emotion'] = le.fit_transform(df['Emotion']) 
y = df.pop('Emotion')
#print(y)
y_new = tf.keras.utils.to_categorical(y, num_classes=7)
print(y_new)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
list(le.inverse_transform([0,1,2,3,4,5,6]))

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [ ]:
y.value_counts()

3    15193
4     6436
6     3459
0     2629
5     2152
1      714
2      532
Name: Emotion, dtype: int64

In [ ]:
# 2 options to handle imbalanced dataset: class_weight or focal loss
class_weight = {0: 6, 1: 22, 2: 30, 3: 1, 4: 2, 5: 7, 6: 5}

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y_new, test_size=0.05, stratify=y_new)

x_val = x_train[:100]
y_val = y_train[:100]
x_train = x_train[100:]
y_train = y_train[100:]

In [ ]:
from tensorflow.keras import activations

def focal_loss(gamma=2., alpha=4., from_logits=False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax if from_logits is False.
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed


In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)
input_layer = Input(shape=(max_length), )
x = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_layer)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x_a = GlobalMaxPool1D()(x)
x_b = GlobalAveragePooling1D()(x)
x = concatenate([x_a,x_b])
x = Dense(64, activation="relu")(x)
x = Dense(7, activation='softmax')(x)
model_w2v = Model(inputs=input_layer, outputs=x)
#model_ft.compile(loss=focal_loss(alpha=1), optimizer='adam', metrics=['accuracy']) #use if you are using focal loss
model_w2v.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #use if you are using class_weight
model_w2v.summary()

NameError: ignored

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_w2v,show_shapes= True)

In [ ]:
model_w2v.fit(x_train, y_train, epochs = 50, validation_data=(x_val, y_val), callbacks=[callback], class_weight=class_weight) #insert class_weight=class_weight if using class_weight

Epoch 1/50
921/921 [==============================] - 80s 82ms/step - loss: 5.7351 - accuracy: 0.4416 - val_loss: 1.4754 - val_accuracy: 0.5300
Epoch 2/50
921/921 [==============================] - 75s 81ms/step - loss: 4.9423 - accuracy: 0.5444 - val_loss: 1.4899 - val_accuracy: 0.5300
Epoch 3/50
921/921 [==============================] - 74s 81ms/step - loss: 4.5998 - accuracy: 0.5699 - val_loss: 1.7247 - val_accuracy: 0.5100
Epoch 4/50
921/921 [==============================] - 74s 81ms/step - loss: 4.2927 - accuracy: 0.5889 - val_loss: 1.5139 - val_accuracy: 0.5300
Epoch 5/50
921/921 [==============================] - 75s 81ms/step - loss: 3.9992 - accuracy: 0.6022 - val_loss: 1.6021 - val_accuracy: 0.5000
Epoch 6/50
921/921 [==============================] - 75s 81ms/step - loss: 3.6981 - accuracy: 0.6168 - val_loss: 1.5112 - val_accuracy: 0.5100
Epoch 7/50
921/921 [==============================] - 74s 81ms/step - loss: 3.4322 - accuracy: 0.6285 - val_loss: 1.6736 - val_accuracy:

In [ ]:
y_pred = model_w2v.predict(x_test)

In [ ]:
y_pred = np.argmax(y_pred, 1)
y_test = np.argmax(y_test, 1)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.30      0.34      0.32       131
           1       0.25      0.39      0.31        36
           2       0.03      0.04      0.04        26
           3       0.85      0.77      0.81       760
           4       0.50      0.51      0.50       322
           5       0.40      0.44      0.42       108
           6       0.53      0.55      0.54       173

    accuracy                           0.61      1556
   macro avg       0.41      0.43      0.42      1556
weighted avg       0.64      0.61      0.62      1556

